In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using PyPlot
using SparseArrays
using ControlSystems
using ForwardDiff

In [ ]:
# Discrete dynamics
h = 0.1   # time step
A = [1 h; 0 1]
B = [0.5*h*h; h]

In [ ]:
#Controllability
rank([B A*B])

In [ ]:
n = 2     # number of state
m = 1     # number of controls
Tfinal = 10.0 # final time
N = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(N-1), step=h));

In [ ]:
# Initial conditions
x0 = [1.0; 0]

In [ ]:
# Cost weights
Q = Array(1.0*I(n))
R = Array(0.1*I(m))
Qn = Array(1.0*I(n))

In [ ]:
#Cost function
function J(xhist,uhist)
    cost = 0.5*xhist[:,end]'*Qn*xhist[:,end]
    for k = 1:(size(xhist,2)-1)
        cost = cost + 0.5*xhist[:,k]'*Q*xhist[:,k] + 0.5*(uhist[k]'*R*uhist[k])[1]
    end
    return cost
end

In [ ]:
#Cost-to-go Function
function Vinf(x)
    return 0.5*x'*Pinf*x
end

function V(k,x)
    return 0.5*x'*P[:,:,k]*x
end

In [ ]:
#QP Solution for xhist, uhist
# Cost
H = blockdiag(sparse(R), kron(I(N-2), blockdiag(sparse(Q),sparse(R))), sparse(Qn));

In [ ]:
# Constraints
C = kron(I(N-1), [B -I(2)])
for k = 1:N-2
    C[(k*n).+(1:n), (k*(n+m)-n).+(1:n)] .= A
end
d = [-A*x0; zeros(size(C,1)-n)];

In [ ]:
# Solve the linear system
y = [H C'; C zeros(size(C,1),size(C,1))]\[zeros(size(H,1)); d]

# Get multipliers
λhist_qp = reshape(y[(size(H,1)+1):end],n,N-1)

# Get state history
z = y[1:size(H,1)]   # states and controls [u0,x1,u1,...,xN]
Z = reshape(z,n+m,N-1)
xhist_qp = Z[m+1:end,:]
xhist_qp = [x0 xhist_qp]

# Get control history
uhist_qp = Z[1,:];

In [ ]:
#Dynamic Programming Solution for P and K
P = zeros(n,n,N)
K = zeros(m,n,N-1)

P[:,:,N] .= Qn

#Backward Riccati recursion
for k = (N-1):-1:1
    K[:,:,k] .= (R .+ B'*P[:,:,k+1]*B)\(B'*P[:,:,k+1]*A)
    P[:,:,k] .= Q + K[:,:,k]'*R*K[:,:,k] + (A-B*K[:,:,k])'*P[:,:,k+1]*(A-B*K[:,:,k])
end

#Forward rollout starting at x0
xhist = zeros(n,N)
xhist[:,1] = x0
uhist = zeros(m,N-1)
for k = 1:(N-1)
    uhist[:,k] .= -K[:,:,k]*xhist[:,k]
    xhist[:,k+1] .= A*xhist[:,k] + B*uhist[k]
end

In [ ]:
# Plot x1 vs. x2, u vs. t, x vs. t, etc.
times = range(0,h*(N-1), step=h)
plot(times,xhist[1,:], label="DP position")
plot(times,xhist[2,:], label="DP velocity")
plot(times,xhist_qp[1,:], label="QP position")
plot(times,xhist_qp[2,:], label="QP velocity")
xlabel("time")
legend()

In [ ]:
plot(times[1:end-1], uhist[1,:], label="DP control")
plot(times[1:end-1], uhist_qp, label="QP control")
xlabel("Time")
legend()

In [ ]:
plot(P[1,1,:])
plot(P[1,2,:])
plot(P[2,2,:])

In [ ]:
J(xhist,uhist)

In [ ]:
J(xhist_qp,uhist_qp)

In [ ]:
#Compute infinite-horizon K matrix using ControlSystems.jl
Kinf = dlqr(A,B,Q,R[1])
#Compare to ours
K[:,:,1]-Kinf

In [ ]:
#Compute infinite-horizon P matrix
Pinf = dare(A,B,Q,R)
#Compare to ours
P[:,:,1] - Pinf

In [ ]:
#Forward rollout starting at xk
function rollout(k,x)
    xsub = zeros(n,N-k+1)
    xsub[:,1] .= x
    usub = zeros(m,N-k)
    for j = k:(N-1)
        usub[:,j-k+1] .= -K[:,:,j]*xsub[:,j-k+1]
        xsub[:,j-k+2] .= A*xsub[:,j-k+1] + B*usub[j-k+1]
    end
    return xsub,usub
end

In [ ]:
#Generate a sub-trajectory starting at xk
k = 50
xsub, usub = rollout(k,xhist[:,k])

In [ ]:
#Optimal state sub-trajectories are optimal
xsub - xhist[:,k:end]

In [ ]:
#Optimal control sub-trajectories are optimal
usub - uhist[:,k:end]

In [ ]:
#Compare multipliers from QP to Cost-to-go gradient from DP
λhist_qp[:,k-1]

In [ ]:
ForwardDiff.gradient(x->V(k,x),xhist[:,k])

In [ ]:
#Also compare to infinite horizon
ForwardDiff.gradient(x->Vinf(x),xhist[:,k])

In [ ]:
#Let's try finite diffing the cost w.r.t. the state
x1p, u1p = rollout(k,xhist[:,k]+[1e-6; 0])
x2p, u2p = rollout(k,xhist[:,k]+[0; 1e-6])
λfd = [J(x1p,u1p) - J(xhist[:,k:end],uhist[:,k:end]), J(x2p,u2p) - J(xhist[:,k:end],uhist[:,k:end])]./1e-6